In [16]:
"""
Franke surface + language-shaped prior inside a Bayesian Optimization loop
===========================================================================

What this script does
---------------------
- Defines a 2D synthetic objective (Franke function) on [0,1]^2 to **maximize**.
- Uses your **language_shaped_prior_llm.py** helpers (Schema, PriorBuilder, etc.).
- Fits a **GP on residuals** (y - m0(x)), then wraps it with a deterministic prior mean m0(x)
  so the acquisition sees the **total posterior = m0(x) + GP_residual(x)**.
- Runs an **analytic EI** loop (q=1) and tracks best-so-far.
- Provides example readouts (flat, helpful, misleading) so you can compare how priors change performance.
- Jupyter-friendly: functions return pandas DataFrames; one-liner demo at bottom.

Usage (in Jupyter)
------------------
1) Ensure `language_shaped_prior_llm.py` is importable (we add '/mnt/data' to sys.path by default).
2) Run this cell/file; then run the demo cell at the very bottom (or call `compare_readouts(...)`).
3) Optional: call `plot_runs(history_df)` to visualize best-so-far.

Dependencies
------------
- torch >= 1.11, botorch, gpytorch, numpy, pandas, matplotlib

Notes
-----
- We use **analytic ExpectedImprovement** (EI) so we don't need fantasies.
- The wrapper model ensures EI evaluates improvement on the **total** posterior (m0 + residual GP),
  which is the principled way to use a shaped prior with BO.
"""

from __future__ import annotations
import sys, math, warnings
from dataclasses import dataclass
from typing import Dict, Any, List, Tuple, Optional

# --- Make sure your helper module is reachable ---
sys.path.append('/mnt/data')  # adjust if needed

import numpy as np
import pandas as pd

import torch
from torch import Tensor

torch.set_default_dtype(torch.double)

a = torch.tensor(1.0)  # just to ensure torch is imported

# BoTorch / GPyTorch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.optim.optimize import optimize_acqf
from botorch.models.model import Model
from botorch.posteriors.gpytorch import GPyTorchPosterior
from botorch.utils.transforms import normalize
from botorch.utils.sampling import draw_sobol_samples

import gpytorch
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.distributions import MultivariateNormal

# Your prior helpers
from language_shaped_prior_llm import (
    Schema, ContinuousVar, CategoricalVar,
    PriorBuilder, HeuristicReadout
)

# -------------------- Synthetic objective: Franke surface --------------------

def franke_function_np(xy: np.ndarray) -> np.ndarray:
    """Franke function on [0,1]^2. Accepts shape (N,2) numpy, returns shape (N,).
    We will **maximize** it (no sign flip). No noise here; add outside if needed.
    """
    x = xy[:, 0]
    y = xy[:, 1]
    term1 = 0.75 * np.exp(-(9*x - 2)**2 / 4.0 - (9*y - 2)**2 / 4.0)
    term2 = 0.75 * np.exp(-(9*x + 1)**2 / 49.0 - (9*y + 1)**2 / 10.0)
    term3 = 0.50 * np.exp(-(9*x - 7)**2 / 4.0 - (9*y - 3)**2 / 4.0)
    term4 = 0.20 * np.exp(-(9*x - 4)**2 - (9*y - 7)**2)
    return term1 + term2 + term3 - term4


def true_objective(X: Tensor, noise_sd: float = 0.0, rng: Optional[np.random.Generator] = None) -> Tensor:
    """Torch wrapper around Franke + optional Gaussian noise.
    X: (N,2) in [0,1]
    Returns (N,1)
    """
    xy = X.detach().cpu().numpy()
    f = franke_function_np(xy)
    if noise_sd > 0.0:
        if rng is None:
            rng = np.random.default_rng()
        f = f + rng.normal(0.0, noise_sd, size=f.shape)
    return torch.as_tensor(f, dtype=X.dtype, device=X.device).unsqueeze(-1)


# ---------------- Language-shaped prior tooling (Schema + PriorBuilder) -------

def make_schema() -> Schema:
    # You can change roles to trigger different default heuristics in HeuristicReadout
    return Schema(
        continuous=[
            ContinuousVar(name="x1", low=0.0, high=1.0, role="temperature"),
            ContinuousVar(name="x2", low=0.0, high=1.0, role="concentration"),
        ],
        categorical=[],
    )


def make_readout_flat(schema: Schema) -> Dict[str, Any]:
    """A neutral prior (m0 = 0 everywhere)."""
    effects = {v.name: {"effect": "flat", "scale": 0.0, "confidence": 0.0} for v in schema.continuous}
    return {"effects": effects, "interactions": []}


def make_readout_helpful(schema: Schema) -> Dict[str, Any]:
    """A helpful prior: both x1 and x2 with nonmonotone peaks + positive synergy.
    This roughly matches Franke's bumpy landscape and should accelerate BO.
    """
    effects = {
        "x1": {"effect": "nonmonotone-peak", "scale": 0.8, "confidence": 0.9, "range_hint": [0.3, 0.7]},
        "x2": {"effect": "nonmonotone-peak", "scale": 0.8, "confidence": 0.9, "range_hint": [0.2, 0.8]},
    }
    return {
        "effects": effects,
        "interactions": [
            {"pair": ["x1", "x2"], "type": "synergy", "confidence": 0.7}
        ],
    }


def make_readout_very_helpful(schema: Schema) -> Dict[str, Any]:
    """A very informative prior with narrow peaks (strong guidance).
    You can tweak the ranges; narrower hints make the prior more directive.
    """
    effects = {
        "x1": {"effect": "nonmonotone-peak", "scale": 1.0, "confidence": 0.98, "range_hint": [0.55, 0.65]},
        "x2": {"effect": "nonmonotone-peak", "scale": 1.0, "confidence": 0.98, "range_hint": [0.15, 0.25]},
    }
    return {
        "effects": effects,
        "interactions": [
            {"pair": ["x1", "x2"], "type": "synergy", "confidence": 0.9}
        ],
    }


def make_readout_misleading(schema: Schema) -> Dict[str, Any]:
    """A misleading prior (e.g., says x1 decreases, x2 increases monotonically)."""
    effects = {
        "x1": {"effect": "decrease", "scale": 0.7, "confidence": 0.8},
        "x2": {"effect": "increase", "scale": 0.7, "confidence": 0.8},
    }
    return {"effects": effects, "interactions": []}


# ------------------ GP over residuals + deterministic mean wrapper ------------

class GPWithPriorMean(Model):
    """Wrap a residual GP with a deterministic prior mean m0(x) from PriorBuilder.

    The base GP is fit on residuals: y_resid = y - m0(x).
    The posterior returned by this wrapper is the total posterior of f(x) = m0(x) + g(x),
    where g ~ GP_residual. Covariance is unchanged.
    """
    def __init__(self, base_gp: SingleTaskGP, prior: PriorBuilder, schema: Schema, m0_scale: float = 1.0):
        super().__init__()
        self.base_gp = base_gp
        self.prior = prior
        self.schema = schema
        self.m0_scale = float(m0_scale)

    @property
    def num_outputs(self) -> int:
        return 1

    def _m0_from_X(self, X: Tensor) -> Tensor:
        # X may be shaped (..., q, d). Flatten to 2D, compute m0 via PriorBuilder, then reshape back.
        d = X.shape[-1]
        X2 = X.reshape(-1, d).detach().cpu().numpy()
        cols = [v.name for v in self.schema.continuous] + [c.name for c in self.schema.categorical]
        df = pd.DataFrame(X2, columns=cols)
        m0_np = self.prior.m0_df(df)  # shape (N,)
        m0 = torch.as_tensor(self.m0_scale * m0_np, dtype=X.dtype, device=X.device).reshape(X.shape[:-1])
        return m0

    def posterior(self, X: Tensor, observation_noise: bool = False, **kwargs) -> GPyTorchPosterior:
        base_post = self.base_gp.posterior(X, observation_noise=observation_noise, **kwargs)
        mvn = base_post.mvn
        m0 = self._m0_from_X(X).reshape(mvn.mean.shape)
        new_mvn = MultivariateNormal(mean=mvn.mean + m0, covariance_matrix=mvn.covariance_matrix)
        return GPyTorchPosterior(new_mvn)

    # Delegate commonly-used utilities to base model (defensive completeness)
    def condition_on_observations(self, X: Tensor, Y: Tensor, noise: Optional[Tensor] = None, **kwargs):
        cm = self.base_gp.condition_on_observations(X=X, Y=Y, noise=noise, **kwargs)
        return GPWithPriorMean(base_gp=cm, prior=self.prior, schema=self.schema, m0_scale=self.m0_scale)

    def fantasize(self, X: Tensor, sampler, observation_noise: bool = True, **kwargs):
        fm = self.base_gp.fantasize(X=X, sampler=sampler, observation_noise=observation_noise, **kwargs)
        return GPWithPriorMean(base_gp=fm, prior=self.prior, schema=self.schema, m0_scale=self.m0_scale)

    def subset_output(self, idcs: List[int]) -> 'GPWithPriorMean':
        # Single output; just return self.
        return self


# ------------------------------ BO utilities ---------------------------------

def fit_residual_gp(X_obs: Tensor, y_obs: Tensor, prior: PriorBuilder, schema: Schema) -> Tuple[SingleTaskGP, Tensor, float]:
    """Fit SingleTaskGP on residuals y - alpha*m0(X) with a calibrated scale alpha.
    Returns (residual_gp, y_resid, alpha).
    X_obs: (N,2) in [0,1]; y_obs: (N,1)
    """
    # Compute raw m0 on the observed points
    df_obs = pd.DataFrame(X_obs.detach().cpu().numpy(), columns=[v.name for v in schema.continuous])
    m0_raw_np = prior.m0_df(df_obs)  # (N,)
    y_np = y_obs.detach().cpu().numpy().reshape(-1)

    # Calibrate a scale alpha so prior magnitude matches the data magnitude.
    # Use least-squares with intercept -> center both vectors.
    m0_c = m0_raw_np - m0_raw_np.mean()
    y_c = y_np - y_np.mean()
    denom = float(np.dot(m0_c, m0_c) + 1e-12)
    alpha = float(np.dot(m0_c, y_c) / denom) if denom > 0 else 0.0

    # Residuals
    m0_t = torch.as_tensor(alpha * m0_raw_np, dtype=X_obs.dtype, device=X_obs.device).unsqueeze(-1)
    y_resid = y_obs - m0_t

    gp = SingleTaskGP(train_X=X_obs, train_Y=y_resid)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)
    return gp, y_resid, alpha


def propose_next_point(model_total: Model, X_dim: int = 2, num_restarts: int = 10, raw_samples: int = 256) -> Tensor:
    bounds = torch.stack([torch.zeros(X_dim, dtype=torch.double), torch.ones(X_dim, dtype=torch.double)])
    # Use analytic EI (q=1)
    # best_f will be set when constructing EI by caller, since it depends on y_obs
    # Here we only optimize the acqf once constructed.
    def _optimize(acqf):
        cand, _ = optimize_acqf(
            acq_function=acqf,
            bounds=bounds,
            q=1,
            num_restarts=num_restarts,
            raw_samples=raw_samples,
            options={"maxiter": 200, "batch_limit": 5},
        )
        return cand.squeeze(0)  # (2,)
    return _optimize


def bo_loop(
    readout: Dict[str, Any],
    n_init: int = 4,
    n_iter: int = 20,
    noise_sd: float = 0.0,
    seed: int = 0,
    prior_strength_pseudopts: float = 8.0,
    alpha_boost: float = 1.0,
    alpha_decay: float = 1.0,
    verbose: bool = False,
) -> pd.DataFrame:
    """Run one BO loop with a given readout. Returns a history DataFrame with:
    iter, x1, x2, y, best_so_far, acq, readout_name
    """
    rng = np.random.default_rng(seed)
    torch.manual_seed(seed)

    schema = make_schema()
    prior = PriorBuilder(schema=schema, readout=readout, prior_strength_pseudopts=prior_strength_pseudopts)

    # Initial design via Sobol
    X_init = draw_sobol_samples(bounds=torch.stack([torch.zeros(2), torch.ones(2)]), n= n_init, q=1, seed=seed).squeeze(1)
    y_init = true_objective(X_init, noise_sd=noise_sd, rng=rng)

    X_obs = X_init.clone()
    y_obs = y_init.clone()

    records: List[Dict[str, Any]] = []

    def _record(i: int, x: Tensor, y: Tensor, best: float):
        records.append({
            "iter": i,
            "x1": float(x[0].item()),
            "x2": float(x[1].item()),
            "y": float(y.item()),
            "best_so_far": float(best),
        })

    # Record initials
    best = float(y_obs.max().item())
    for i in range(X_obs.shape[0]):
        _record(i - X_obs.shape[0], X_obs[i], y_obs[i], best)

    # Optimization iterations
    for t in range(n_iter):
        # 1) fit residual GP on current data
        gp_resid, y_resid, alpha = fit_residual_gp(X_obs, y_obs, prior, schema)
        alpha_eff = alpha * alpha_boost * (alpha_decay ** t)
        model_total = GPWithPriorMean(gp_resid, prior, schema, m0_scale=alpha_eff)

        # 2) build EI on total posterior
        best_f = float(y_obs.max().item())
        EI = ExpectedImprovement(model=model_total, best_f=best_f, maximize=True)

        # 3) optimize EI for next point
        bounds = torch.stack([torch.zeros(2, dtype=torch.double), torch.ones(2, dtype=torch.double)])
        x_next, _ = optimize_acqf(
            acq_function=EI,
            bounds=bounds,
            q=1,
            num_restarts=12,
            raw_samples=256,
            options={"maxiter": 200, "batch_limit": 5},
        )
        x_next = x_next.squeeze(0)

        # 4) evaluate objective
        y_next = true_objective(x_next.unsqueeze(0), noise_sd=noise_sd, rng=rng).squeeze(0)

        # 5) update
        X_obs = torch.cat([X_obs, x_next.unsqueeze(0)], dim=0)
        y_obs = torch.cat([y_obs, y_next.unsqueeze(0)], dim=0)
        best = max(best, float(y_next.item()))
        _record(t, x_next, y_next, best)

        if verbose and ((t+1) % 5 == 0 or t == n_iter - 1):
            print(f"Iter {t+1:02d}: y_next={y_next.item():.4f}, best={best:.4f}")

    hist = pd.DataFrame.from_records(records)
    return hist


def compare_readouts(
    readouts: Dict[str, Dict[str, Any]],
    n_init: int = 4,
    n_iter: int = 20,
    noise_sd: float = 0.0,
    seed: int = 0,
    prior_strength_pseudopts: float = 8.0,
    alpha_boost: float = 1.0,
    alpha_decay: float = 1.0,
) -> pd.DataFrame:
    """Run multiple readouts and concatenate history with a 'readout' column."""
    dfs = []
    for name, ro in readouts.items():
        df = bo_loop(
            readout=ro,
            n_init=n_init,
            n_iter=n_iter,
            noise_sd=noise_sd,
            seed=seed,
            prior_strength_pseudopts=prior_strength_pseudopts,
            alpha_boost=alpha_boost,
            alpha_decay=alpha_decay,
        )
        df["readout"] = name
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)


def benchmark_readouts(
    readouts: Dict[str, Dict[str, Any]],
    seeds: List[int],
    n_init: int = 4,
    n_iter: int = 20,
    noise_sd: float = 0.0,
    prior_strength_pseudopts: float = 8.0,
    alpha_boost: float = 1.0,
    alpha_decay: float = 1.0,
) -> pd.DataFrame:
    dfs = []
    for s in seeds:
        df = compare_readouts(
            readouts,
            n_init=n_init,
            n_iter=n_iter,
            noise_sd=noise_sd,
            seed=s,
            prior_strength_pseudopts=prior_strength_pseudopts,
            alpha_boost=alpha_boost,
            alpha_decay=alpha_decay,
        )
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)


# ------------------------------ Plotting -------------------------------------

def estimate_franke_max(grid_n: int = 201) -> float:
    xs = np.linspace(0,1,grid_n)
    yy, xx = np.meshgrid(xs, xs)
    pts = np.c_[xx.ravel(), yy.ravel()]
    vals = franke_function_np(pts)
    return float(vals.max())


def aggregate_by_seed(history_df: pd.DataFrame) -> pd.DataFrame:
    # assumes columns: iter, best_so_far, readout, seed (we will add seed)
    return (history_df
            .groupby(["readout", "iter"], as_index=False)
            .agg(mean_best=("best_so_far", "mean"), std_best=("best_so_far", "std")))


def plot_runs(history_df: pd.DataFrame):
    import matplotlib.pyplot as plt
    plt.figure()
    for name, grp in history_df.groupby("readout"):
        g = grp.copy()
        g = g[g["iter"] >= 0]
        plt.plot(g["iter"].values, g["best_so_far"].values, label=name)
    plt.xlabel("Iteration")
    plt.ylabel("Best so far (Franke)")
    plt.title("Bayesian Optimization with Language-Shaped Priors (EI)")
    plt.legend()
    plt.show()


def plot_runs_mean(history_df: pd.DataFrame):
    import matplotlib.pyplot as plt
    agg = aggregate_by_seed(history_df)
    plt.figure()
    for name, grp in agg.groupby("readout"):
        g = grp.copy()
        g = g[g["iter"] >= 0]
        plt.plot(g["iter"].values, g["mean_best"].values, label=name)
        if not g["std_best"].isna().all():
            lo = g["mean_best"].values - g["std_best"].values
            hi = g["mean_best"].values + g["std_best"].values
            plt.fill_between(g["iter"].values, lo, hi, alpha=0.2)
    plt.xlabel("Iteration")
    plt.ylabel("Best so far (mean ± sd)")
    plt.title("BO with Priors — averaged over seeds")
    plt.legend()
    plt.show()


# # ------------------------------- Demo cell -----------------------------------
# if __name__ == "__main__":
#     schema = make_schema()
#     readouts = {
#         "flat": make_readout_flat(schema),
#         "helpful": make_readout_helpful(schema),
#         "misleading": make_readout_misleading(schema),
#     }
#     hist = compare_readouts(readouts, n_init=8, n_iter=30, noise_sd=0.0, seed=0)
#     print(hist.tail())
#     # To visualize inside Jupyter: plot_runs(hist)


In [ ]:
schema = make_schema()
readouts = {
    "flat": make_readout_flat(schema),
    "helpful": make_readout_helpful(schema),
    "very_helpful": make_readout_very_helpful(schema),  # narrower, stronger hints
    "misleading": make_readout_misleading(schema),
}

# Stress the early, low-data regime so priors matter
df = benchmark_readouts(
    readouts,
    seeds=list(range(10)),
    n_init=3,           # fewer initials -> rely more on prior
    n_iter=15,          # short budget -> priors show up early
    alpha_boost=2.5,    # make prior ~2.5x stronger initially
    alpha_decay=0.98    # slowly anneal down each iter
)

plot_runs_mean(df)  # mean ± sd across seeds

# (Optional) quantify in regret against approx global max
fmax = estimate_franke_max()
df["regret"] = fmax - df["best_so_far"]
aggregate_by_seed(df)[["readout","iter"]].assign(
    mean_regret=(fmax - aggregate_by_seed(df)["mean_best"])
)

c:\Users\amirr\anaconda3\envs\torch_env\lib\site-packages\botorch\acquisition\analytic.py:337: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)
c:\Users\amirr\anaconda3\envs\torch_env\lib\site-packages\botorch\acquisition\analytic.py:337: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)
c:\Users\amirr\anaconda3\envs\torch_env\lib\site